**Author:** Kajetan Bochajczuk

*Importing of all necessary libraries and tuning some of the global settings.*

In [1]:
import numpy as np  # For numerical calculations
import matplotlib 
import matplotlib.pyplot as plt  # For plotting
import pandas as pd  # For reading external data and displaying it in tabular form

In [2]:
matplotlib.rcParams['figure.figsize'] = [12, 10]
# matplotlib.rcParams['figure.dpi'] = 600
matplotlib.rc('font', family='Lato', size=12)

# Exercise 1
Cnsider the data from file `data-lab5.xlsx`, which consist of observations of two variables $[Y, X]$. Suppose, $Y$ is generated by the process
$$
y_i = \alpha_0 + \alpha_1 x_i + \alpha_2 x_i^2 + e_i
$$
where $e_i$ is a white noise and is assumed to follow a normal distribution $e_i \sim \mathcal{N}(0, \sigma^2)$.
- Verify a hypothesis $H_0: \sigma^2 = 1$ versus the alternative $H_1: \sigma^2 > 1$ with Wald test. Present formula for the appropriate test statistic.
- Suppose, you want to test, if the polynomial $\alpha_0 + \alpha_1 x_i + \alpha_2 x_i^2$ has exactly one root. Derive and present the test for Wald statistic. What is its asymptotic distribution? Present the result.

# Exercise 2
Suppos, you want to verify if in the model $y_t = \alpha x_t + e_t$ the parameter $\alpha$ is different from zero. The hypothesis is typically tested with a t-Student test. The aim of this exercise is to check, how important is the stationarity assumptions of residuals. In order to verify the test performance, propose and conduct a small Monte-Carlo experiment.
- Define the numberf of MC iterations ($N$) and the sample size (choose one sample size, for example $T=200$).
- Choose values of the parameter $\alpha$, for which the frequency of rejections will be computed (both under the null and the alternative).
- Generate $N$ different realizations of the stationary exogenous variable $x_t$ (it may come from a normal distribution $x_t \sim \mathcal{N}(\mu_x, \sigma_x^2)$).
- Describe the test properties (size and power), for stationary residuals. In order to do so, for each MC iteration, generate a white noise $e_t \sim \mathcal{N}(0, \sigma^2)$. Then, conditional on the parameter $\alpha$, generate $y_t$ and compute the $t$ statistic. Compare it with the critical value from the t-Student distribution and check if the null is rejected or not.
- Now, repeat the procedure from previous ppoint for residuals, which come form the random walk.
- Compute the frequency of null rejections for different scenarios and parameter values.
- What are the empirical size and powers of the test, when the residuals are white noise/random walk. Comment on the results.